In [1]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import altair as alt
from requests.utils import quote
import os
from datetime import timedelta
from mod import alt_theme

In [2]:
fmt = "{:%Y-%m-%d}"

# Can optionally use number of days to choose dates
n_days = 60
end_date = fmt.format(pd.datetime.today())
start_date = fmt.format(pd.datetime.today() - timedelta(days=n_days))

renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]
bot_names = ["stale", "codecov", "jupyterlab-dev-mode", "henchbot"]

In [3]:
# Parameters
renderer = "kaggle"
start_date = "2019-01-01"
end_date = "2019-02-01"


In [4]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [5]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [6]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [7]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [8]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [9]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (106 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (60 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (25 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (29 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [What&rsquo;s an alternative to the word &ldquo;Fellow&rdquo; or &ldquo;Fellowship&rdquo;?](https://discourse.jupyter.org/t/whats-an-alternative-to-the-word-fellow-or-fellowship) (16 posts)
* [Microsoft Word Integration (Intern Project)](https://discourse.jupyter.org/t/microsoft-word-integration-intern-project) (19 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (14 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [Jupyter community workshop: proposal on diversity, inclusion and accessibility](https://discourse.jupyter.org/t/jupyter-community-workshop-proposal-on-diversity-inclusion-and-accessibility) (20 posts)

## Active topics on the Community Forum

These are topics with lots of activity in recent history.

In [10]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (106 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (60 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (52 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (31 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (29 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (25 posts)
* [Potential collaboration on user research?](https://discourse.jupyter.org/t/potential-collaboration-on-user-research) (21 posts)

In [11]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:            category_id  bookmarked     parent_category  \
  11       accessibility           2      Special Topics   
  10      Admin requests           1       Site Feedback   
  0                 Meta           4                Meta   
  1        announcements           1                Meta   
  8           JupyterLab           1          JupyterLab   
  6              discuss           3          JupyterHub   
  2           JupyterHub           2          JupyterHub   
  9   Enterprise Gateway           2  Enterprise Gateway   
  3       Community Chat           3      Community Chat   
  7              discuss           6              Binder   
  4               Binder           4              Binder   
  5            BinderHub           1              Binder   
  
                            parent/category  
  11           Special Topics/accessibility  
  10           Site Feedback/Admin requests  
  0                               Meta/Meta  
  1                      Meta/announcements  
  8                   JupyterLab/JupyterLab  
  6                      JupyterHub/discuss  
  2                   JupyterHub/JupyterHub  
  9   Enterprise Gateway/Enterprise Gateway  
  3           Community Chat/Community Chat  
  7                          Binder/discuss  
  4                           Binder/Binder  
  5                        Binder/BinderHub  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['accessibility', 'Admin requests', 'Meta', 'announcements', 'JupyterLab', 'discuss', 'JupyterHub', 'Enterprise Gateway', 'Community Chat', 'discuss', 'Binder', 'BinderHub']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [12]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [User interface tweaks on mybinder.org](https://discourse.jupyter.org/t/user-interface-tweaks-on-mybinder-org) (9 posts)
* [JupyterHub Community Call, August 2019](https://discourse.jupyter.org/t/jupyterhub-community-call-august-2019) (7 posts)
* [Teaching Jupyter to people without computing experience](https://discourse.jupyter.org/t/teaching-jupyter-to-people-without-computing-experience) (7 posts)
* [Accessibility Audit for Jupyter Notebook](https://discourse.jupyter.org/t/accessibility-audit-for-jupyter-notebook) (6 posts)
* [Multiple Classes Multiple Jupyterhubs/labs same domain?](https://discourse.jupyter.org/t/multiple-classes-multiple-jupyterhubs-labs-same-domain) (5 posts)
* [Jupyter Source code debugging](https://discourse.jupyter.org/t/jupyter-source-code-debugging) (5 posts)
* [GitHub issues/PRs - lock bot?](https://discourse.jupyter.org/t/github-issues-prs-lock-bot) (5 posts)
* [NBConvert 5.6.0 Release](https://discourse.jupyter.org/t/nbconvert-5-6-0-release) (4 posts)
* [Download multiple files from notebook server?](https://discourse.jupyter.org/t/download-multiple-files-from-notebook-server) (4 posts)
* [JupyterHub LDAP server](https://discourse.jupyter.org/t/jupyterhub-ldap-server) (4 posts)

In [13]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:                          category_id  bookmarked parent_category  \
  10                     accessibility           1  Special Topics   
  7                                Q&A           3             Q&A   
  11                          Notebook           0        Notebook   
  1                               jobs           2            Meta   
  2                               Meta           1            Meta   
  6                         JupyterLab           1      JupyterLab   
  3                         JupyterHub           2      JupyterHub   
  0   Zero to JupyterHub on Kubernetes           1      JupyterHub   
  9                               team           1      JupyterHub   
  4                     Community Chat           0  Community Chat   
  5                             Binder           1          Binder   
  8                   mybinder.org ops           1          Binder   
  
                                  parent/category  
  10                 Special Topics/accessibility  
  7                                       Q&A/Q&A  
  11                            Notebook/Notebook  
  1                                     Meta/jobs  
  2                                     Meta/Meta  
  6                         JupyterLab/JupyterLab  
  3                         JupyterHub/JupyterHub  
  0   JupyterHub/Zero to JupyterHub on Kubernetes  
  9                               JupyterHub/team  
  4                 Community Chat/Community Chat  
  5                                 Binder/Binder  
  8                       Binder/mybinder.org ops  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['accessibility', 'Q&A', 'Notebook', 'jobs', 'Meta', 'JupyterLab', 'JupyterHub', 'Zero to JupyterHub on Kubernetes', 'team', 'Community Chat', 'Binder', 'mybinder.org ops']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## User activity in the Community Forum

**Top posters**

These people have posted lots of comments, replies, answers, etc in the community forum.

In [14]:
def plot_user_data(users, column, sort=False):
    plt_data = users.sort_values(column, ascending=False).head(50)
    x = alt.X("username", sort=plt_data['username'].tolist()) if sort is True else 'username'
    ch = alt.Chart(data=plt_data).mark_bar().encode(
        x=x,
        y=column
    )
    return ch

In [15]:
url = "https://discourse.jupyter.org/directory_items.json"
params = {"period": "quarterly", "order": "post_count"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
users = pd.DataFrame(resp.json()['directory_items'])
users['username'] = users['user'].map(lambda a: a['username'])

In [16]:
plot_user_data(users.head(50), 'post_count')

Chart({
  data:      id  likes_received  likes_given  topics_entered  topic_count  post_count  \
  0     6             129          206             330           11         135   
  1     3             120          196             230           30         123   
  2    24              46           16             264            6          63   
  3     8              64           13             140            4          51   
  4   220              36            1              95            1          50   
  5    12              37            7             122            6          38   
  6   116              59           29             127            4          37   
  7    73              18           17             329            2          27   
  8    99              13           37              97            3          22   
  9   485              20           57              58            1          21   
  10   11              14           66             208            1          20   
  12  234              16            5               7            4          17   
  11  217               6           24              35            0          17   
  13   77              10            8             153            0          13   
  14   30              13           71              50            2          12   
  15   60               7            7              33            0          11   
  16   34              18            3              16            3          10   
  17  222              15            6              16            1          10   
  18  432               6           24              51            1          10   
  19  566               2            0              12            2           9   
  20  556               8            3              15            1           9   
  21  679              10            1              24            0           9   
  22  303               3            7              67            2           9   
  23    7              16           24              31            0           9   
  26  686               3            0              30            3           8   
  25  419               2            0              14            5           8   
  24  510               6            0               4            2           8   
  28  462               1            0              16            3           7   
  29  268              13            5              18            3           7   
  30  390               2            1              14            4           7   
  31   59              10            3              16            1           7   
  32  372               6            5              27            1           7   
  33   71               4            1              15            1           7   
  34  333               0            2               1            0           7   
  27  487               0            0              25            4           7   
  40  227               2            7              28            0           6   
  43  642               1            0              30            5           6   
  42  590               2            0              13            2           6   
  41  771              11            1               3            0           6   
  38  565               5            3               6            0           6   
  39  145               8            0               7            2           6   
  37    9               6            0              10            0           6   
  36  640               0            0               6            1           6   
  35  195               1            0               8            0           6   
  44  562               0            0               4            0           5   
  45  352               7            0               3            0           5   
  46  699               1            0              14            5           5   
  

**Forum users, sorted by likes given**

These are Community Forum members that "liked" other people's posts. We appreciate
anybody taking the time to tell someone else they like what they're shared!

In [17]:
plot_user_data(users.head(50), 'likes_given')

Chart({
  data:      id  likes_received  likes_given  topics_entered  topic_count  post_count  \
  0     6             129          206             330           11         135   
  1     3             120          196             230           30         123   
  14   30              13           71              50            2          12   
  10   11              14           66             208            1          20   
  9   485              20           57              58            1          21   
  8    99              13           37              97            3          22   
  6   116              59           29             127            4          37   
  23    7              16           24              31            0           9   
  11  217               6           24              35            0          17   
  18  432               6           24              51            1          10   
  7    73              18           17             329            2          27   
  2    24              46           16             264            6          63   
  3     8              64           13             140            4          51   
  49  184               2           11              37            1           5   
  13   77              10            8             153            0          13   
  47  622               4            8              17            1           5   
  22  303               3            7              67            2           9   
  40  227               2            7              28            0           6   
  15   60               7            7              33            0          11   
  5    12              37            7             122            6          38   
  17  222              15            6              16            1          10   
  29  268              13            5              18            3           7   
  32  372               6            5              27            1           7   
  12  234              16            5               7            4          17   
  31   59              10            3              16            1           7   
  38  565               5            3               6            0           6   
  16   34              18            3              16            3          10   
  20  556               8            3              15            1           9   
  34  333               0            2               1            0           7   
  33   71               4            1              15            1           7   
  21  679              10            1              24            0           9   
  30  390               2            1              14            4           7   
  4   220              36            1              95            1          50   
  41  771              11            1               3            0           6   
  45  352               7            0               3            0           5   
  46  699               1            0              14            5           5   
  48  601               1            0              39            3           5   
  44  562               0            0               4            0           5   
  43  642               1            0              30            5           6   
  42  590               2            0              13            2           6   
  36  640               0            0               6            1           6   
  39  145               8            0               7            2           6   
  37    9               6            0              10            0           6   
  24  510               6            0               4            2           8   
  35  195               1            0               8            0           6   
  19  566               2            0              12            2           9   
  28  462               1            0              16            3           7   
  

**Forum users, sorted by likes received**

These are folks that posted things other people in the Community Forum liked.

In [18]:
plot_user_data(users.head(50), 'likes_received')

Chart({
  data:      id  likes_received  likes_given  topics_entered  topic_count  post_count  \
  0     6             129          206             330           11         135   
  1     3             120          196             230           30         123   
  3     8              64           13             140            4          51   
  6   116              59           29             127            4          37   
  2    24              46           16             264            6          63   
  5    12              37            7             122            6          38   
  4   220              36            1              95            1          50   
  9   485              20           57              58            1          21   
  16   34              18            3              16            3          10   
  7    73              18           17             329            2          27   
  23    7              16           24              31            0           9   
  12  234              16            5               7            4          17   
  17  222              15            6              16            1          10   
  10   11              14           66             208            1          20   
  29  268              13            5              18            3           7   
  14   30              13           71              50            2          12   
  8    99              13           37              97            3          22   
  41  771              11            1               3            0           6   
  31   59              10            3              16            1           7   
  21  679              10            1              24            0           9   
  13   77              10            8             153            0          13   
  20  556               8            3              15            1           9   
  39  145               8            0               7            2           6   
  15   60               7            7              33            0          11   
  45  352               7            0               3            0           5   
  32  372               6            5              27            1           7   
  18  432               6           24              51            1          10   
  24  510               6            0               4            2           8   
  37    9               6            0              10            0           6   
  11  217               6           24              35            0          17   
  38  565               5            3               6            0           6   
  47  622               4            8              17            1           5   
  33   71               4            1              15            1           7   
  26  686               3            0              30            3           8   
  22  303               3            7              67            2           9   
  42  590               2            0              13            2           6   
  40  227               2            7              28            0           6   
  25  419               2            0              14            5           8   
  30  390               2            1              14            4           7   
  19  566               2            0              12            2           9   
  49  184               2           11              37            1           5   
  35  195               1            0               8            0           6   
  28  462               1            0              16            3           7   
  43  642               1            0              30            5           6   
  46  699               1            0              14            5           5   
  48  601               1            0              39            3           5   
  36  640               0            0               6            1           6   
  

In [19]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>